### In this final assignment your goal is to process a large corpus of unstructured chat data, discover it's structure and build a Question Answering system based on it

In [1]:
import re
import os
import json
import nltk
import scipy
import pymystem3
import numpy as np
import pandas as pd

from nltk.tag import pos_tag
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models import KeyedVectors
from itertools import chain
from tqdm import tqdm

### 3.1 Unzip and explore the data

In [2]:
# !unzip ./support_logs.zip

In [5]:
faq = open("./support_faq.txt", encoding='utf8').read()

In [6]:
raw_chats = open("./support_chats.txt", encoding='utf8').read()

In [7]:
# print(faq[:995])

In [8]:
def change_med(med_data):
    
    client_reply = med_data['client_reply']
    personal_reply = med_data['personal_reply']

    concat_client = []
    concat_personal = []
    
    summary = []

    if len(client_reply) != 1:
        for n,repl in enumerate(client_reply):
            if n == 0:
                client = repl[0]
            else:
                if repl[2] == client_reply[n-1][2]:
                    client += ' '
                    client += repl[0]
                else:
                    concat_client.append(client)
                    client = repl[0]
        concat_client.append(client)
    else:
        concat_personal = [personal_reply[0][2]]
        
    if len(personal_reply) != 1:
        for n,repl in enumerate(personal_reply):
            if n == 0:
                personal = repl[0]
            else:
                if repl[1] == personal_reply[n-1][1]:
                    personal += ' '
                    personal += repl[0]
                else:
                    concat_personal.append(personal)
                    personal = repl[0]
        concat_personal.append(personal)
    else:
        concat_personal = [personal_reply[0][2]]
        
    if len(concat_personal) > len(concat_client):
        for i in range(len(concat_personal) - len(concat_client)):
            concat_client.append('')
    elif len(concat_personal) < len(concat_client):
        for i in range(len(concat_client) - len(concat_personal)):
            concat_personal.append('')
    
    for v in range(len(concat_personal)):
        summary.append([concat_client[v], concat_personal[v]])
        
    return summary

In [9]:
data = []
for chat in tqdm(raw_chats.split('-- -- -- --')):
    client = 0
    personal = 0
    chat_id = -1
    client_id = -1
    med_data = {'client_reply':[], 'personal_reply':[]}
    for string in chat.split('\n'):
        try:
            if string.find('Chat number') != -1:
                chat_id = string.split('=')[1].strip()
            elif string.find('Client Id') != -1:
                client_id = string.split('=')[1].strip()
            elif string.find('Клиент') != -1:
                med_data['client_reply'].append([string.split('Клиент :')[1].strip(), client, personal])
                client += 1
            elif string.find('Сотрудник') != -1:
                med_data['personal_reply'].append([string.split('Сотрудник :')[1].strip(), client, personal])
                personal += 1
        except:
            pass
    try:
        for d in change_med(med_data):
            data.append([client_id, chat_id, d[0], d[1]])
    except:
        print(med_data, chat)

 22%|██▏       | 6178/28363 [00:00<00:01, 15434.38it/s]

{'client_reply': [], 'personal_reply': []}  Пересылаемое сообщение 
{'client_reply': [], 'personal_reply': []}  Пересылаемое сообщение 


100%|██████████| 28363/28363 [00:01<00:00, 14228.51it/s]

{'client_reply': [], 'personal_reply': []} 




In [10]:
dialogues = pd.DataFrame(data, columns=['client_id', 'chat_id', 'question', 'answer'])

In [14]:
lemmatizer = pymystem3.Mystem()
def tokenize_and_lemmatize(text):
    analysis = lemmatizer.analyze(text.strip())
    tokens = []
    for an in analysis:
        if 'analysis' in an:
            try:
                tokens.append(an['analysis'][0]['lex'])
            except IndexError:
                tokens.append(an['text'])
    return ' '.join(tokens)

In [15]:
dialogues['token_question'] = [tokenize_and_lemmatize(i) for i in tqdm(dialogues.question)]
dialogues['token_answer'] = [tokenize_and_lemmatize(i) for i in tqdm(dialogues.answer)]


 16%|█▌        | 26915/171017 [00:15<01:20, 1780.57it/s]


 32%|███▏      | 55171/171017 [00:29<01:01, 1884.21it/s]


 49%|████▉     | 83403/171017 [00:43<00:45, 1912.72it/s]


 65%|██████▌   | 111522/171017 [00:57<00:30, 1928.69it/s]


 81%|████████▏ | 138952/171017 [01:11<00:16, 1932.19it/s]


...

 62%|██████▏   | 105661/171017 [01:23<00:51, 1263.42it/s]


 72%|███████▏  | 123184/171017 [01:37<00:37, 1258.88it/s]


 82%|████████▏ | 140260/171017 [01:51<00:24, 1252.93it/s]


 92%|█████████▏| 157733/171017 [02:05<00:10, 1252.53it/s]


100%|██████████| 171017/171017 [02:16<00:00, 1253.18it/s]

In [16]:
dialogues.to_csv('diag.csv',sep=';',index=False)

In [17]:
# dialogues[dialogues.question.str.find('?')==-1][:10]

In [18]:
# dialogues[dialogues.question.str.find('?')!=-1].shape, dialogues[dialogues.question.str.find('?')==-1].shape

In [19]:
# dialogues.answer.nunique()

In [20]:
# dialogues[dialogues.question.str.find('?')!=-1][:10]

In [21]:
# tfidf = TfidfVectorizer()

In [22]:
# tfidf.fit(list(dialogues.question) + list(dialogues.answer))

In [23]:
# x = tfidf.transform(dialogues.question)

In [24]:
# res = []
# for i in range(5,50):
#     print(i)
#     km = KMeans(n_clusters=i)
#     km.fit_predict(x)
#     res.append(np.sqrt(km.inertia_))

array([0, 0, 0, ..., 0, 0, 0])

### The data consists of 2 files. The first one contains raw logs of some bank's customer support. The second one contains frequently asked questions with answers. 

### 3.2 Building a structured FAQ

### A good place to start would be to turn the FAQ into a more structured form, then find paraphrases for each question from the chat logs.

In [25]:
faq_qna = []
buf = []
for line in faq.split("\n"):
    if line == '' and len(buf):
        faq_qna.append(buf)
        buf = []
    else:
        buf.append(line)
        
FAQ = []
for faq_entry in faq_qna:
    faq_dict = {}
    faq_dict['answer'] = faq_entry[1]
    faq_dict['question'] = faq_entry[0]
    faq_dict['paraphrased_questions'] = []
    FAQ.append(faq_dict)

In [35]:
faq_data = []
for i in tqdm(FAQ):
    faq_data.append([tokenize_and_lemmatize(i['question']), tokenize_and_lemmatize(i['answer'])])


100%|██████████| 171/171 [00:00<00:00, 555.37it/s]

In [36]:
pd.DataFrame(faq_data,columns=['question', 'answer']).to_csv('faq.csv',sep=';',index=False)

In [32]:
faq_train = json.load(open("./faq_train.json"))
faq_test = json.load(open("./faq_val.json"))

In [37]:
faq_data = []
for i in tqdm(faq_train):
    faq_data.append([tokenize_and_lemmatize(i['question']), tokenize_and_lemmatize(i['answer'])])
    for j in i['paraphrased_questions']:
        faq_data.append([tokenize_and_lemmatize(j), tokenize_and_lemmatize(i['answer'])])
    
pd.DataFrame(faq_data,columns=['question', 'answer']).to_csv('faq_test.csv',sep=';',index=False)

faq_data = []
for i in tqdm(faq_test):
    faq_data.append([tokenize_and_lemmatize(i['question']), tokenize_and_lemmatize(i['answer'])])
    for j in i['paraphrased_questions']:
        faq_data.append([tokenize_and_lemmatize(j), tokenize_and_lemmatize(i['answer'])])
    
pd.DataFrame(faq_data,columns=['question', 'answer']).to_csv('faq_val.csv',sep=';',index=False)


100%|██████████| 30/30 [00:01<00:00, 26.87it/s]

100%|██████████| 30/30 [00:00<00:00, 49.27it/s]

In [46]:
FAQ

[{'answer': 'Да , если банкомат поддерживает данную функцию .',
  'paraphrased_questions': [],
  'question': '1 . Могу ли я поменять ПИН-код своей карты <bankname> Банка в банкомате ?'},
 {'answer': 'Расплачиваться картой и снимать наличные можно в любой валюте .',
  'paraphrased_questions': [],
  'question': '2 . В каких валютах можно расплачиваться картой <bankname> Банка ?'},
 {'answer': 'Да , возможно . Для этого вам необходимо в магазине , в котором вы приобретаете товар , попросить предоставить дополнительный чек Tax Free . На данный чек необходимо поставить штамп на таможне . Возврат средств по чекам Tax Free возможен в международном аэропорту ( в стране , где совершалась покупка ) в специальном пункте возврата НДС .',
  'paraphrased_questions': [],
  'question': '3 . При оплате товаров за границей , я оплачиваю НДС в казну другого государства , возможно ли вернуть эти средства ?'},
 {'answer': 'Получать выписку по ссылке в письме так же безопасно , как получать выписку в виде в

### From now on, you are on your own. 

![architecture](showme.png)

### 3.3 Find paraphrases for existing questions

### 3.4 Find clusters of frequent questions